<a href="https://colab.research.google.com/github/mrzaid/dl-cnn-bootcamp/blob/main/cnnKerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner
import tensorflow as tf
from tensorflow import keras
import numpy as np

data=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=data.load_data()
# scaling down to bost up the process
train_images=train_images/255.0
test_images=test_images/255.0
train_images[0].shape
# 4d input value do cnn ko
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [3]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    # no of neurons to be select
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # last layer with 10 outputs
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [4]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9144999980926514

Best val_accuracy So Far: 0.9144999980926514
Total elapsed time: 00h 03m 09s


In [6]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        27712     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 112)               4128880   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 4,158,202
Trainable params: 4,158,202
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 5ms/step - loss: 0.1130 - accuracy: 0.9580 - val_loss: 0.2522 - val_accuracy: 0.9202
Epoch 5/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0751 - accuracy: 0.9724 - val_loss: 0.2940 - val_accuracy: 0.9163
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0481 - accuracy: 0.9836 - val_loss: 0.3342 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0367 - accuracy: 0.9865 - val_loss: 0.3889 - val_accuracy: 0.9177
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0265 - accuracy: 0.9904 - val_loss: 0.4689 - val_accuracy: 0.9135
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0198 - accuracy: 0.9927 - val_loss: 0.4420 - val_accuracy: 0.9142
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0196 - accuracy: 0.9936 - val_loss: 0.5225 - val_accura